# REDUIRE lES DIMENSIONS ET IMAGE

In [ ]:
# Les données

# La vidéo dure 1 h 36, et contient 174 053 images. Une image est composée de
# pixels. 
# Chaque image mesure 640 pixels en largeur et 360 en hauteur, soit 230 400 
# pixels ! En plus, la couleur d'un pixel est encodée selon 3 niveaux : rouge, 
# vert et bleu. On a donc 174 053 * 230 400 * 3 = 120 305 433 600 unités
# d'information dans cette vidéo ! C'est un peu lourd pour nos traitements 
# statistiques, on devra donc réduire la taille des données.

# Pour en réduire la taille, nous ne sélectionnons qu'une image sur 90 (une 
# toutes les 3 secondes). Ensuite, on réduit la taille des images à 72 pixels
# de hauteur et 128 de largeur. Nous réduisons également la palette des 
# couleurs possibles : originellement, les niveaux des 3 couleurs (RGB) sont 
# encodés sur une échelle de 0 à 256 : nous réduirons cette échelle à une 
# échelle de 0 à 64.

# On obtient donc un tableau où chaque ligne (chaque individu) correspond à 1 
# image. Il y a 1 934 images. Chaque image est caractérisée par 72*128*3 = 
# 27 648 unités d'information, donc autant de colonnes. On ajoute en plus une 
# colonne qui indique la position de l'image dans le temps (par exemple, 
# "cette image s'affiche à la150e seconde").

In [ ]:
import imageio
import PIL
from PIL import Image
import numpy as np
import pandas as pd

In [ ]:
# Paramètres de compression
width, height = 128,72
img_freq = 90

In [ ]:
# Import de la video
filename = 'ACP.mp4'
images_per_seconds = 30 # La vidÃ©o dÃ©file Ã  30 images par secondes
url = "https://youtu.be/uV5hmpzmWsU"
vid = imageio.get_reader(filename,  'ffmpeg')
sample = np.arange(0, vid.get_length(), img_freq)

In [ ]:
# Extraction des informations
first_frame = vid.get_data(0)
image_height, image_width, colors = first_frame.shape
print(image_height,image_width,vid.get_length(),colors)
print(image_height * image_width * vid.get_length() * colors)



In [ ]:
# Extraction des images
video = np.zeros((0, width*height*3), dtype=np.int8)
for j,i in enumerate(sample):
    if j%5 == 0:
        print("image ",i, max(sample))
    image = vid.get_data(i)

    # Reduction de la taille des images
    img = Image.fromarray(image.astype('uint8'), 'RGB')
    img = img.resize((width,height), PIL.Image.ANTIALIAS)
    image = np.array(img)
    
    image = image.reshape(height*width*3)
    image = image / 4 #Reduction de l'Ã©chelle des couleurs (0 Ã  256 vers du 0 Ã  64)
    image = np.array(image, dtype=np.int8) # Conversion des niveaux de couleurs en int
    #display_image(image,height,width,3)
    video = np.concatenate([video,[image]])

print(video.shape)



In [ ]:
# conversion en dataframe
video = pd.DataFrame(video)



In [ ]:
# ajout de la position de l'image, en secondes :
video["position"] = [int(frame/images_per_seconds) for frame in sample]
# ajout de l'url qui fait dÃ©marrer la vidÃ©o au bon endroit
video.index = ["{}?t={}s".format(url, p) for p in video["position"]]
# inversion de l'ordre des colonnes dans le dataframe
cols = list(video.columns)
cols.pop(cols.index('position'))
video = video[['position'] + cols]

# Export CSV
video.to_csv("video.csv")